In [1]:
import os 
import cv2   
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import numpy as np  
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from IPython.display import SVG

C:\Users\lms2755\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [ ]:
#Load the data
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    class_labels = {'buildings': 0,
                    'forest' : 1,
                    'glacier' : 2,
                    'mountain' : 3,
                    'sea' : 4,
                    'street' : 5
                    }
    for label in os.listdir(directory):
        
        for image_file in os.listdir(directory+label): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+label+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(class_labels[label])
    
    return shuffle(Images,Labels) #Shuffle the dataset you just prepared.


In [ ]:
Images, Labels = get_images('./seg_train/') #Extract the training images from the folders.

X_train = np.array(Images, dtype = 'float32') #converting the list of images to numpy array.
y_train = np.array(Labels, dtype = 'int32')

X_train = X_train/(X_train.max()) #Scale data



In [ ]:

def get_classlabel(class_code):
    labels = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
    return labels[class_code]

f,ax = plt.subplots(5,5) 
f.subplots_adjust(0,0,3,3)
for i in range(0,5,1):
    for j in range(0,5,1):
        rnd_number = random.randint(0,len(X_train))
        ax[i,j].imshow(X_train[rnd_number])
        ax[i,j].set_title(get_classlabel(y_train[rnd_number]))
        ax[i,j].axis('off')


In [ ]:
Images, Labels = get_images('./seg_test/') #Extract the training images from the folders.

X_test = np.array(Images, dtype = 'float32') #converting the list of images to numpy array.
y_test = np.array(Labels, dtype = 'int32')

X_test = X_test/(X_train.max()) #Scale data


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), # the nn will learn the good filter to use
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.5), # To deal with over fitting
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

model.compile(optimizer=Optimizer.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split = 0.3)
score = model.evaluate(X_test, y_test, batch_size=128)
score